In [ ]:
import os
import pandas as pd
from pathlib import Path
from transformers import pipeline
import re

/home/kennym1/.conda/envs/MA384/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_rows', 1000)

In [ ]:
# # Path to Downloads (Windows example)
# downloads_path = Path.home() / "Downloads"

# # Collect file info
# data = []
# for file in downloads_path.iterdir():
#     if file.is_file():
#         stat = file.stat()
#         data.append({
#             "name": file.name,
#             "size_bytes": stat.st_size,
#             "suffix": file.suffix,
#             "date_modified": pd.to_datetime(stat.st_mtime, unit="s").strftime("%Y-%m-%d"),
#             "date_created": pd.to_datetime(stat.st_birthtime, unit="s").strftime("%Y-%m-%d"),
#         })

# # Convert to DataFrame
# df = pd.DataFrame(data)

# print(df.head())


                                      name  size_bytes suffix date_modified  \
0  +----------------------------------.txt        3112   .txt    2025-07-10   
1                                .Rhistory        1297           2025-12-22   
2                            ._ipp2-source         178           2022-09-07   
3      01 PCProcessorMicroarchitecture.pdf      170784   .pdf    2025-03-13   
4               01-probability-review.docx       19858  .docx    2025-12-09   

  date_created  
0   2025-07-10  
1   2025-12-22  
2   2025-11-04  
3   2025-03-13  
4   2025-12-09  


In [7]:
df = pd.read_csv("/home/kennym1/download_analysis/downloads_info.csv")
print(df.head())

                                      name  size_bytes suffix date_modified  \
0  +----------------------------------.txt        3112   .txt    2025-07-10   
1                                .Rhistory        1297    NaN    2025-12-22   
2                            ._ipp2-source         178    NaN    2022-09-07   
3      01 PCProcessorMicroarchitecture.pdf      170784   .pdf    2025-03-13   
4               01-probability-review.docx       19858  .docx    2025-12-09   

  date_created  
0   2025-07-10  
1   2025-12-22  
2   2025-11-04  
3   2025-03-13  
4   2025-12-09  


In [ ]:
# df.to_csv("downloads_info.csv", index=False)

In [8]:
print(df.suffix.value_counts())

suffix
.pdf                                        570
.docx                                        68
.pptx                                        67
.png                                         45
.html                                        37
.txt                                         26
.csv                                         22
.sql                                         17
.jpg                                         14
.jfif                                        11
.xlsx                                        10
.bmpr                                         9
.jpeg                                         8
.json                                         6
.ans                                          6
.in                                           6
.py                                           6
.v                                            5
.java                                         5
.ipynb                                        4
.log                             

In [ ]:
# --- Stage 1: Rule-based by extension (instant, no model needed) ---
EXTENSION_RULES = {
    # Software
    ".exe": "Software Installer", ".msi": "Software Installer",
    ".iso": "Software Installer", ".dmg": "Software Installer",
    ".apk": "Software Installer", ".deb": "Software Installer",
    ".msix": "Software Installer", ".appinstaller": "Software Installer",
    ".whl": "Software Installer",
    # Media
    ".mp4": "Media or Entertainment", ".mkv": "Media or Entertainment",
    ".avi": "Media or Entertainment", ".mp3": "Media or Entertainment",
    ".wav": "Media or Entertainment", ".flac": "Media or Entertainment",
    ".ogg": "Media or Entertainment",
    # Images
    ".jpg": "Photo or Image", ".jpeg": "Photo or Image",
    ".jfif": "Photo or Image",
    ".png": "Photo or Image",  ".gif": "Photo or Image",
    ".svg": "Photo or Image",  ".psd": "Creative Project",
    # Code / Data
    ".py": "Dataset or Code", ".js": "Dataset or Code",
    ".csv": "Dataset or Code", ".json": "Dataset or Code",
    ".ipynb": "Dataset or Code", ".sql": "Dataset or Code",
    ".c": "Dataset or Code", ".cpp": "Dataset or Code",
    ".java": "Dataset or Code", ".v": "Dataset or Code",
    ".tsv": "Dataset or Code",
    # Archives
    ".zip": "Archive or Backup", ".rar": "Archive or Backup",
    ".tar": "Archive or Backup", ".gz": "Archive or Backup",
    ".7z": "Archive or Backup", ".xz": "Archive or Backup",
    ".bak": "Archive or Backup", ".bac": "Archive or Backup",
    ".tmp": "Archive or Backup",
}

# --- Stage 2: Keyword rules applied to the cleaned filename stem ---
# More keywords = fewer files need the slow NLP model
KEYWORD_RULES = {
    "Schoolwork": [
        "lecture", "homework", "assignment", "exam", "quiz", "lab",
        "midterm", "final", "chapter", "tutorial", "worksheet", "syllabus",
        "notes", "slide", "textbook", "problem set", "pset",
        # common CS/stats course topics
        "probability", "statistics", "algorithm", "microarchitecture",
        "hypothesis", "estimation", "likelihood", "regression", "prediction",
        "processor", "architecture", "operating system", "compiler",
        "database", "network", "machine learning", "data structure",
        "sorting", "search", "graph", "tree", "recursion",
        "prior", "posterior", "bayesian",
    ],
    "Research Paper": [
        "survey", "review", "study", "analysis", "evaluation",
        "proceedings", "conference", "acm", "ieee", "arxiv",
        "journal", "abstract", "methodology", "experiment",
    ],
    "Financial": [
        "invoice", "receipt", "bank", "statement", "tax", "budget",
        "payroll", "expense", "payment", "billing", "account",
    ],
    "Personal Documentation": [
        "resume", "cv", "cover letter", "passport", "license",
        "certificate", "transcript", "contract", "agreement", "lease",
        "insurance", "medical", "prescription",
    ],
}

# --- Stage 3: NLP zero-shot for files that pass through keyword stage ---
AMBIGUOUS_EXTENSIONS = {".pdf", ".docx", ".doc", ".txt", ".pptx", ".xlsx",
                        ".xls", ".html", ".rtf", ".tex", ".latex", ".md"}

NLP_LABELS = [
    "Personal Documentation",
    "Schoolwork or Course Material",
    "Financial Record",
    "Research Paper or Academic Article",
    "Creative Project",
    "Dataset or Code",
    "Archive or Backup",
]


def clean_for_nlp(name: str) -> str:
    """
    Produce a human-readable description from a raw filename so the
    zero-shot model has something meaningful to classify.
    """
    stem = Path(name).stem
    # Strip leading numeric prefix: "01 ", "02-", "03. ", etc.
    stem = re.sub(r'^\d+[\s.\-_]+', '', stem)
    # Strip UUID-like tokens (they add noise, not signal)
    stem = re.sub(r'\b[0-9a-f]{8}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{4}-[0-9a-f]{12}\b', '', stem, flags=re.I)
    # Split CamelCase: "PCProcessorMicroarchitecture" → "PC Processor Microarchitecture"
    stem = re.sub(r'([a-z])([A-Z])', r'\1 \2', stem)
    stem = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1 \2', stem)
    # Replace common separators with spaces
    stem = stem.replace('_', ' ').replace('-', ' ').replace('.', ' ')
    # Collapse multiple spaces
    stem = re.sub(r'\s+', ' ', stem).strip()
    return stem


def keyword_classify(cleaned_stem: str) -> str | None:
    """Return a label if any keyword matches, else None (needs NLP)."""
    lower = cleaned_stem.lower()
    for label, keywords in KEYWORD_RULES.items():
        if any(kw in lower for kw in keywords):
            return label
    return None


In [10]:
classifier = pipeline(
        "zero-shot-classification",
        model="cross-encoder/nli-deberta-v3-small",
        device=0,  # remove this line if you don't have a GPU
)

Loading weights: 100%|██████████| 106/106 [00:00<00:00, 368.18it/s, Materializing param=pooler.dense.weight]                                     
DebertaV2ForSequenceClassification LOAD REPORT from: cross-encoder/nli-deberta-v3-small
Key                             | Status     |  | 
--------------------------------+------------+--+-
deberta.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
def classify_with_rules(row) -> tuple[str | None, str]:
    """Returns (category, cleaned_name). category is None if NLP is needed."""
    ext = str(row["suffix"]).lower() if pd.notna(row["suffix"]) else ""
    name = str(row["name"])
    cleaned = clean_for_nlp(name)

    # Stage 1: hard extension rules
    if ext in EXTENSION_RULES:
        return EXTENSION_RULES[ext], cleaned

    # Stage 2: keyword match on cleaned stem (fast, no model)
    if ext in AMBIGUOUS_EXTENSIONS:
        kw_label = keyword_classify(cleaned)
        if kw_label:
            return kw_label, cleaned
        return None, cleaned  # falls through to NLP

    return "Other", cleaned

results_tuple = df.apply(classify_with_rules, axis=1, result_type="expand")
df["category"] = results_tuple[0]
df["cleaned_name"] = results_tuple[1]

needs_nlp = df["category"].isna()
print(f"Extension rules : {(df['category'].notna() & df['category'].ne('Other')).sum()} files")
print(f"Keyword rules   : {needs_nlp.sum()} still need NLP")
print(f"Total needing NLP: {needs_nlp.sum()} / {len(df)}")

if needs_nlp.any():
    texts = df.loc[needs_nlp, "cleaned_name"].tolist()

    nlp_results = classifier(
        texts,
        candidate_labels=NLP_LABELS,
        # More natural framing for document filenames — avoids the generic
        # "This example is X" default which inflates Archive/Backup scores
        hypothesis_template="This file is a {}.",
        batch_size=16,
    )

    df.loc[needs_nlp, "category"] = [r["labels"][0] for r in nlp_results]

print("\nSample results:")
print(df[["name", "suffix", "cleaned_name", "category"]].head(25).to_string(index=True))
